In [1]:
import pandas as pd
import numpy as np
from itertools import product
import copy
from tqdm import tqdm

In [2]:
df= pd.DataFrame({
    "N_candidat": ["0001", "0002", "0003", "0004", "0005"],
    "S1":[15, 11, 14, 16, 11],
    "S2": [13, 5, 12, 15, 11],
    "S3":[16, 8, 11, 15, 11],
    "S4": [17, 12, 15, 18, 11],
    "S5": [13, 10, 13, 15, 11],

    "Logique": [1, 1, 1, 0, 0],
    "BDD": [1, 1, 1, 0, 0],
    "PS": [1, 1, 1, 1, 0],
    "Moyenne": [2, 0, 1, 2, 0],
    "Eligible": [True, True, True, False, False],
    "Decision": ['A','R','L','R','R']
})
V = list(df.columns)[1:-1]
display(V, df)

['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS', 'Moyenne', 'Eligible']

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


# ACi
## AC1

In [3]:
#ligne existe, la décision est bonne et toutes les variables de X ont les bonnes valeurs aussi
def AC1(N, X, decision, df=df):
    c = df[df.N_candidat == N].copy()
    return len(c) == 1 and  c.Decision.iloc[0] == decision and all((str(c.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X)


In [4]:
def test_AC1():
    display(df)
    # Cas de succès
    assert AC1("0001", {"Logique": 1}, 'A')
    assert AC1("0001", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    assert AC1("0004", {"S2":15, "BDD":0}, 'R')

    # Cas d'échec
    assert not AC1("xxxx", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    display("AC1 est bien correcte.")

test_AC1()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


'AC1 est bien correcte.'

## AC2

In [5]:
coeffs= [1, 1, 1, 1, 1] #calcul de la Moyenne
seuils = [12, 12, 12] # Calcul du status d'éligibilité (logique, BDD, PS)
thresholds = [12, 14] # prendre la décision

def med(S1, S2, S3, S4, S5, Logique, BDD, PS, coeffs=coeffs, seuils=seuils, thresholds2=thresholds):
    moyenne = (S1 * coeffs[0] + S2 * coeffs[1] + S3 * coeffs[2] + S4 * coeffs[3] + S5 * coeffs[4]) / sum(coeffs)
    e = (Logique*BDD*PS==1)
    m = (moyenne>thresholds2[1])*2 + (thresholds2[0] <= moyenne <= thresholds2[1])
    if e and m==2:
        d= 'A'
    elif e and m==1:
        d='L'
    else:
        d='R'
    return m, e, d

In [6]:
def AC2(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    vals=candidat.iloc[0].to_dict()
    X_var=list(X.keys())

    #W=list(set(candidat.columns)-set(X_var)) #optionnel

    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }
    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]

    combinations = generate_combinations(X, variable_domains)

    # Vérifier chaque combinaison
    for combination in combinations:
        x_prime = dict(zip(X_var, combination))
        for k in x_prime.keys():
            vals[k]=x_prime[k]
        m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

        if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
            continue
        if d!=decision:
            #print(f"AC2 est vérifié pour une affectation x_prime: {x_prime}")
            return True
    return False

In [7]:
def test_AC2():
	display(df)
	# Cas de succès
	assert AC2("0001", {"S1": 15},'A')
	assert AC2("0002", {"S1": 13, "S5": 11}, 'R')
	assert AC2("0003", {"Logique": 1, "S5": 11, "S2":8},'L')

	# Cas d'échec
	assert not AC2("0004", {"S5": 11, "BDD": 0},'R')
	assert not AC2("0005", {"S1": 13, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC2 est bien correcte


In [24]:
def AC2_beta(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }

    def subdict_combinations(dictionary):
        if not dictionary:
            return [{}]

        key = next(iter(dictionary))
        rest = dictionary.copy()
        del rest[key]

        subcombos = subdict_combinations(rest)
        result = []
        for subcombo in subcombos:
            result.append(subcombo)
            with_key = {key: dictionary[key]}
            result.append({**with_key, **subcombo})
        return result

    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]
    sub=subdict_combinations(X)
    for subX in sub:
        if len(subX):
            X_var=list(subX.keys())
            vals=candidat.iloc[0].to_dict()
            combinations = generate_combinations(subX, variable_domains)
            # Vérifier chaque combinaison
            for combination in combinations:
                x_prime = dict(zip(X_var, combination))
                for k in x_prime.keys():
                    vals[k]=x_prime[k]
                m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

                if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
                    continue
                if d!=decision and len(x_prime)==len(X):
                    print(f"AC2 et AC3 est vérifié pour une affectation x_prime: {x_prime}")
                    return True
                elif d!=decision:
                    print(d,decision,x_prime,e,m)
                    return False
    return False
#ca ameliore les cas où AC3 n'est pas vérifié tout en ayant une meme complexité dans le cas où AC2 et AC3 sont verifiés

In [25]:
def test_AC2_beta():
	# Cas de succès
	assert AC2_beta("0001", {"S1": 15},'A')
	assert not AC2_beta("0002", {"S1": 11, "S5": 10}, 'R')
	assert not AC2_beta("0003", {"Logique": 1, "S5": 13, "S2":12},'L')

	# Cas d'échec
	assert not AC2_beta("0004", {"S5": 15, "BDD": 0},'R')
	assert not AC2_beta("0005", {"S1": 11, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



AC2 est bien correcte


In [27]:
def test_AC2():
	# Cas de succès
	assert AC2("0001", {"S1": 15},'A')
	assert AC2("0002", {"S1": 13, "S5": 11}, 'R')
	assert AC2("0003", {"Logique": 1, "S5": 11, "S2":8},'L')

	# Cas d'échec
	assert not AC2("0004", {"S5": 11, "BDD": 0},'R')
	assert not AC2("0005", {"S1": 13, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



AC2 est bien correcte


## AC3:

In [12]:
from itertools import combinations

def AC3(N, X, decision ,df=df):
    keys = list(X.keys())
    subsets = [dict(zip(combination, (X[key] for key in combination))) for r in range(1, len(keys)) for combination in combinations(keys, r)]
    return all(not AC1(N, subset, decision, df) or not AC2(N, subset, decision) for subset in subsets)


In [13]:
def test_AC3():
	display(df)
	# Cas de succès
	assert AC3("0001", {"Logique": 1}, 'A')
	assert AC3("0004", {"Logique": 0, "BDD": 0}, 'R')
	assert AC3("0004", {"Logique": 0, "S1": 16}, 'R')

	# Cas d'échec
	assert not AC3("0001", {"Logique": 1, "PS": 1}, 'A')
	assert not AC3("0004", {"Logique": 0,  "BDD": 0,"S1": 16}, 'R')
	print("AC3 est bien correcte")

test_AC3()


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC3 est bien correcte


In [14]:
def AC(N, X, decision, df=df, afficher = False):
  res1=AC1(N, X, decision, df)
  res2=AC2(N, X, decision, df)
  res3=AC3(N, X, decision, df)
  if afficher:
      print(f"AC1={res1} et AC2={res2} et AC3={res3}")
  return res1, res2, res3

# BCi
## BC1 et BC2

In [15]:
def generate_supersets(N, X, V, df):
	candidat = df[df['N_candidat']==N]
	V_sauf_X=[var for var in V if not var in X.keys()]
	#print(V_sauf_X)
	supersets = [X]
	for var in V_sauf_X:
		valeur = candidat[var].iloc[0]
		copie = copy.deepcopy(supersets)
		for dictio in copie:
			dictio[var]= valeur
		supersets =supersets + copie
	unique_dicts = {tuple(sorted(d.items())) for d in supersets}
	unique_list_of_dicts = [dict(item) for item in unique_dicts]
	return sorted(unique_list_of_dicts, key=lambda x:len(x))

def BC1(N, X, decision, df=df):
	supersets = generate_supersets(N, X[0], V, df)
	for superset in tqdm(supersets):
		#print("\n",superset)
		if all(AC(N, superset, decision[0], df)):
			print(f"La actual cause est: {superset} \n")
			return True
	return False

def BC2(N, X, decision, df=df):
	supersets = generate_supersets(N, X[1], V, df)
	for superset in tqdm(supersets):
		if all(AC(N, superset, decision[1], df)):
			print("La actual cause est: ", superset)
			return True
	return False

Les performances ne sont pas très bonne, en terme de complexité. Nous proposons une version améliorée.

In [28]:
def BC1_beta(N, X, decision, df=df):
	supersets = generate_supersets(N, X[0], V, df)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision[0], df)
		if not ac1:
			return False
		elif not ac2:
			continue
		elif not ac3:
			return False
		print(f"La actual cause est: {superset} \n")
		return True # mon else
	return False

def BC2_beta(N, X, decision, df=df):
	supersets = generate_supersets(N, X[1], V, df)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision[0], df)
		if not ac1:
			return False
		elif not ac2:
			continue
		elif not ac3:
			return False
		print(f"La actual cause est: {superset} \n")
		return True # mon else
	return False

In [29]:
def test_BC1(f=BC1_beta, only_echec = True):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001", ({"S2": 13}, _), ('A', _))
		assert f("0001", ({"Logique": 1}, _), ('A', _))
		assert f("0005", ({"Logique": 0, "BDD": 0, "PS": 0}, _), ('R', _))

	# Cas d'échec
	assert not f("0001", ({"Logique": 1, "BDD": 1}, _), ('A', _)) #AC3
	assert not f("0004", ({"Logique": 1, "BDD": 1, "S1": 16}, _), ('R', _)) #AC1 n'est pas vérifié
	assert not f("0004", ({"Logique": 0, "BDD": 0, "S1": 16}, _), ('R', _)) # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")

In [30]:
######################### CE QUE (RACHA) PROPOSE
def BC1_2_beta(N, X, decision, df=df):
	# verifier que X peut etre une cause partielle
	ac1, ac2, ac3 = AC(N, X, decision, df)
	if not ac1:
		return False
	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause
		return False
	if ac2 and ac3: # si X est une actual cause alors c'est une partial cause
		return True
	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]
	print("Numéro candidat : ",N)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision, df)
		if not ac1:
			continue
		if  ac2 and ac3:
			print(f"La actual cause est: {superset} \n")
			return True

	return False

def test_BC1_beta(f=BC1_2_beta, only_echec = True):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001", {"S2": 13}, 'A')
		assert f("0001", {"Logique": 1}, 'A')
		assert f("0005", {"Logique": 0, "BDD": 0, "PS": 0}, 'R')

	# Cas d'échec
	assert not f("0001", {"Logique": 1, "BDD": 1}, 'A') #AC3
	assert not f("0004", {"Logique": 1, "BDD": 1, "S1": 16}, 'R') #AC1 n'est pas vérifié
	assert not f("0004", {"Logique": 0, "BDD": 0, "S1": 16}, 'R') # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")
test_BC1(only_echec=True)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/128 [00:00<?, ?it/s]

Les tests pour BC1 sont terminés.


In [31]:
test_BC1_beta()
# Nous avons arreté l'éxécution du code


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


Les tests pour BC1 sont terminés.


In [32]:
test_BC1(only_echec=False)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/512 [00:00<?, ?it/s]


La actual cause est: {'S2': 13} 



  0%|          | 0/512 [00:00<?, ?it/s]


La actual cause est: {'Logique': 1} 



  1%|          | 1/128 [00:00<00:14,  8.54it/s]


La actual cause est: {'BDD': 0, 'Logique': 0, 'PS': 0, 'S2': 11} 



  0%|          | 0/128 [00:00<?, ?it/s]

Les tests pour BC1 sont terminés.


## BC3

In [21]:
def BC3(X, df=None):
    X1, X2 = X

    # Check si les deux X1 et X2 contiennent les memes variables
    if set(X1.keys()) != set(X2.keys()):
        return False

    # Check que l'intersection des deux affectations est vide
    for key in X1.keys():
        if X1[key] == X2[key]:
            return False

    return True

In [22]:
def test_BC3():
    # Cas de succès
    assert BC3(({"S2":12, "BDD":0}, {"S2":11, "BDD":1}))

    # Cas d'échec
    assert not BC3(({"S1":15, "BDD":1, "Moyenne":2, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))
    assert not BC3(({"S1":15, "BDD":1, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))

    print("BC3 est bien correcte.")

test_BC3()


BC3 est bien correcte.


## BC4

In [ ]:
def BC4(N_candidats, X, decisions, df=df):
	def BCi1_3(N_candidats, X, decisions, df=df):
		return BC1_2_beta(N_candidats[0], X[0], decisions[0], df) and BC1_2_beta(N_candidats[1], X[1], decisions[1], df) and BC3(X, df)

	# S'assurer que X verifie tous les Bci1-3
	if not BCi1_3(N_candidats, X, decisions, df):
		return False

	candidat1=df[df.N_candidat==N_candidats[0]].copy()
	candidat2=df[df.N_candidat==N_candidats[1]].copy()
	X1=X[0]
	X2=X[1]
	X_var1=list(X1.keys())
	X_var2=list(X2.keys())

	V_var=list(candidat1.columns)[1:-1] # Prendre toutes les variables qui décrivent un candidat1 sauf N_candidat et Décision

	if X_var1 != X_var2 : # les deux candidats doivent etre testés sur les memes attributs (X)
		return False

	X_var=X_var1
	V_minus_X_var=[var for var in V_var if not var in X_var]

	# print(V_minus_X_var)

	# Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
	all_combinaisons_var = []

	for nb_var in range(1, len(V_minus_X_var) + 1):
			all_combinaisons_var.extend(combinations(V_minus_X_var, nb_var))
	#print(all_combinaisons_var)

	for var in tqdm(all_combinaisons_var):
		# Remise à l'état initial des sur-ensemble de X1 et X2
		X1_eval=X1.copy()
		X2_eval=X2.copy()
		for v in var:
			X1_eval.update({v: candidat1.iloc[0, candidat1.columns.get_loc(v)]})
			X2_eval.update({v: candidat2.iloc[0, candidat2.columns.get_loc(v)]})
		X_eval=(X1_eval,X2_eval)
		#print(X_eval)
		if (BCi1_3(N_candidats, X_eval, decisions, df)):
			return False # Retourner False car on aurait trouvé un superset qui verifie tous les BC1-3



	return True
display(df)
BC4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


Numéro candidat :  0004


  1%|          | 4/511 [00:00<00:04, 121.82it/s]


La actual cause est: {'BDD': 0, 'Logique': '0'} 



  0%|          | 0/511 [00:00<?, ?it/s]

100%|█████████▉| 510/511 [08:44<00:01,  1.03s/it] 


MemoryError: 

In [ ]:
def test_BC4():
    # Cas de succès
    display(df)
    assert BC4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))

    # Cas d'échec

    print("BC4 est bien correcte.")

test_BC4()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


Numéro candidat :  0004


  1%|          | 4/511 [00:00<00:03, 148.94it/s]


La actual cause est: {'BDD': 0, 'Logique': '0'} 

[('S1',), ('S2',), ('S3',), ('S4',), ('S5',), ('BDD',), ('PS',), ('Moyenne',), ('Eligible',), ('S1', 'S2'), ('S1', 'S3'), ('S1', 'S4'), ('S1', 'S5'), ('S1', 'BDD'), ('S1', 'PS'), ('S1', 'Moyenne'), ('S1', 'Eligible'), ('S2', 'S3'), ('S2', 'S4'), ('S2', 'S5'), ('S2', 'BDD'), ('S2', 'PS'), ('S2', 'Moyenne'), ('S2', 'Eligible'), ('S3', 'S4'), ('S3', 'S5'), ('S3', 'BDD'), ('S3', 'PS'), ('S3', 'Moyenne'), ('S3', 'Eligible'), ('S4', 'S5'), ('S4', 'BDD'), ('S4', 'PS'), ('S4', 'Moyenne'), ('S4', 'Eligible'), ('S5', 'BDD'), ('S5', 'PS'), ('S5', 'Moyenne'), ('S5', 'Eligible'), ('BDD', 'PS'), ('BDD', 'Moyenne'), ('BDD', 'Eligible'), ('PS', 'Moyenne'), ('PS', 'Eligible'), ('Moyenne', 'Eligible'), ('S1', 'S2', 'S3'), ('S1', 'S2', 'S4'), ('S1', 'S2', 'S5'), ('S1', 'S2', 'BDD'), ('S1', 'S2', 'PS'), ('S1', 'S2', 'Moyenne'), ('S1', 'S2', 'Eligible'), ('S1', 'S3', 'S4'), ('S1', 'S3', 'S5'), ('S1', 'S3', 'BDD'), ('S1', 'S3', 'PS'), ('S1', 'S3', 'Moyenne')

  2%|▏         | 12/511 [00:00<00:04, 114.30it/s]

...

 49%|████▉     | 250/511 [00:02<00:02, 102.98it/s]

({'Logique': '1', 'S4': 17, 'S5': 13, 'BDD': 1, 'Moyenne': 2}, {'Logique': '0', 'S4': 18, 'S5': 15, 'BDD': 0, 'Moyenne': 2})
({'Logique': '1', 'S4': 17, 'S5': 13, 'BDD': 1, 'Eligible': True}, {'Logique': '0', 'S4': 18, 'S5': 15, 'BDD': 0, 'Eligible': False})
({'Logique': '1', 'S4': 17, 'S5': 13, 'PS': 1, 'Moyenne': 2}, {'Logique': '0', 'S4': 18, 'S5': 15, 'PS': 1, 'Moyenne': 2})
({'Logique': '1', 'S4': 17, 'S5': 13, 'PS': 1, 'Eligible': True}, {'Logique': '0', 'S4': 18, 'S5': 15, 'PS': 1, 'Eligible': False})
({'Logique': '1', 'S4': 17, 'S5': 13, 'Moyenne': 2, 'Eligible': True}, {'Logique': '0', 'S4': 18, 'S5': 15, 'Moyenne': 2, 'Eligible': False})
({'Logique': '1', 'S4': 17, 'BDD': 1, 'PS': 1, 'Moyenne': 2}, {'Logique': '0', 'S4': 18, 'BDD': 0, 'PS': 1, 'Moyenne': 2})
({'Logique': '1', 'S4': 17, 'BDD': 1, 'PS': 1, 'Eligible': True}, {'Logique': '0', 'S4': 18, 'BDD': 0, 'PS': 1, 'Eligible': False})
({'Logique': '1', 'S4': 17, 'BDD': 1, 'Moyenne': 2, 'Eligible': True}, {'Logique': '0', '

 51%|█████     | 259/511 [00:06<00:06, 40.38it/s] 

({'Logique': '1', 'S1': 15, 'S2': 13, 'S3': 16, 'S4': 17, 'Eligible': True}, {'Logique': '0', 'S1': 16, 'S2': 15, 'S3': 15, 'S4': 18, 'Eligible': False})


KeyboardInterrupt: 

In [ ]:
# def BC1_2_beta(N, X, decision, df=df):
# 	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]

# 	# verifier que X peut etre une cause partielle
# 	ac1, ac2, ac3 = AC(N, X, decision, df)
# 	if not ac1:
# 		return False
# 	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause
# 		return False

# 	print(N)
# 	for superset in tqdm(supersets):
# 		#print("\n",superset)
# 		ac1, ac2, ac3 = AC(N, superset, decision, df)
# 		if not ac1:
# 			continue
# 		if  ac2 and ac3:
# 			print(f"La actual cause est: {superset} \n")
# 			return True
# 	print("fin------------")
# 	return False


## BC1_4

In [59]:
# ######################### CE QUE (RACHA) PROPOSE avec AC2_beta de Ahmed
# def BC1_2_beta2(N, X, decision, df=df):
# 	# verifier que X peut etre une cause partielle
# 	ac1, ac2, ac3 = AC(N, X, decision, df)
# 	if not ac1:
# 		return False, None
# 	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause
# 		return False, None
# 	if ac2 and ac3: # si X est une actual cause alors c'est une partial cause
# 		return True, None

#   # Créer une structure de données pour enregistrer tous les supersets testés afin d'éviter
# 	dict_var_notAC_candidat={}

# 	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]
# 	print("Numéro candidat : ",N)
# 	for superset in tqdm(supersets):
# 		#print("\n",superset)
# 		if not AC1(N, superset,decision):
# 			return False, dict_var_notAC_candidat
# 		if  AC2_beta(N, superset,decision):
# 			print(f"La actual cause est: {superset} \n")
# 			return True, dict_var_notAC_candidat

# 		dict_var_notAC_candidat.add(superset)

# 	return False, dict_var_notAC_candidat

# def test_BC1_beta2(f=BC1_2_beta2, only_echec = True):
# 	display(df)
# 	# Cas de succès
# 	if not only_echec:
# 		assert f("0001", {"S2": 13}, 'A')[0]
# 		assert f("0001", {"Logique": 1}, 'A')[0]
# 		assert f("0005", {"Logique": 0, "BDD": 0, "PS": 0}, 'R')[0]

# 	# Cas d'échec
# 	assert not f("0001", {"Logique": 1, "BDD": 1}, 'A')[0] #AC3
# 	assert not f("0004", {"Logique": 1, "BDD": 1, "S1": 16}, 'R')[0] #AC1 n'est pas vérifié
# 	assert not f("0004", {"Logique": 0, "BDD": 0, "S1": 16}, 'R')[0] # car S1 n'appartent pas à la actual cause
# 	print("Les tests pour BC1 sont terminés.")
# test_BC1_beta2(only_echec=False)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


Numéro candidat :  0005


  0%|          | 0/127 [00:00<?, ?it/s]

AC2 et AC3 est vérifié pour une affectation x_prime: {'BDD': 1, 'Logique': 1, 'PS': 1, 'S2': 16}
La actual cause est: {'BDD': 0, 'Logique': 0, 'PS': 0, 'S2': 11} 

Les tests pour BC1 sont terminés.


In [ ]:
# def BC1_4(N_candidats, X, decisions, df=df):
#   candidat1=df[df.N_candidat==N_candidats[0]]
#   candidat2=df[df.N_candidat==N_candidats[1]]

#   # Vérifier que X1 de candidat1 et X2 de candidat2 sont testés sur les memes variables (si ce n'est pas vérifié alors BC3 échoue)
#   if X[0].keys()!=X[1].keys():
#     return False

#   Vmax=[]
#   # Générer Vmax : la liste des attributs distincts entre candidat1 et candidat2
#   for col in candidat1.columns[1:-1]:
#     if candidat1.loc[:,[col]].iloc[0,0]!=candidat2.loc[:,[col]].iloc[0,0]:
#       Vmax.append(col)
#   #print(Vmax)

#   # Vérifier si X est inclu dans Vmax (si ce n'est pas vérifié alors BC3 échoue)
#   X_var=[ var for var in X[0]]
#   if not set(X_var).issubset(set(Vmax)):
#     return False

#   # Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
#   all_combinaisons_var = []

#   for nb_var in range(1, len(Vmax) + 1):
#       all_combinaisons_var.extend(combinations(Vmax, nb_var))

#   all_combinaisons_var_reversed=all_combinaisons_var[::-1]
#   #print(all_combinaisons_var_reversed)


#   # Verifier si cet element est partial cause
#   for combinaison in tqdm(all_combinaisons_var_reversed):

#     if not set(X_var).issubset(combinaison) : # On en veut évaluer que les supersets de X
#       continue

#     dict_candidat1=candidat1.loc[:,combinaison].iloc[0].to_dict()
#     dict_candidat2=candidat2.loc[:,combinaison].iloc[0].to_dict()

#     BC1_2_beta_bool1, var_notAC_candidat1=BC1_2_beta2(N_candidats[0], dict_candidat1, decisions[0])
#     if not (BC1_2_beta_bool1):
#       continue

#     BC1_2_beta_bool2, var_notAC_candidat2=BC1_2_beta2(N_candidats[1], dict_candidat2, decisions[1])
#     if not BC1_2_beta_bool2:
#       continue


#     # Verifier la BC4
#     if len(set(X_var)) < len(combinaison):  # BC4 n'est pas vérifiée car nous avons trouver un superset qui verifie BC1_3
#       return False,combinaison

#     else : # len(X_var)==len(combinaison)
#       return True, X_var

#     return False, None # Cas ou c'est un sous ensemble de X_var qui vérifie BC1_4


# BC1_4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))

In [79]:
######################### CE QUE (RACHA) PROPOSE avec AC2_beta de Ahmed et struture de donnée augmentée
def BC1_2_beta2(N, X, decision, var_notAC_candidat=set(), df=df ):
	# verifier que X peut etre une cause partielle
	ac1, ac2, ac3 = AC(N, X, decision, df)
	if not ac1:
		return False, None
	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause
		return False, None
	if ac2 and ac3: # si X est une actual cause alors c'est une partial cause
		return True, None

  # Créer une structure de données pour enregistrer tous les supersets testés afin d'éviter

	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]
	print("Numéro candidat : ",N)
	for superset in tqdm(supersets):

		if tuple(superset.keys()) in var_notAC_candidat: # ignorer tous les supersets (passés en entrée) qui ne verifient pas les ACi
			continue

		if not AC1(N, superset,decision):
			return False, var_notAC_candidat
		if  AC2_beta(N, superset,decision):
			print(f"La actual cause est: {superset} \n")
			return True, var_notAC_candidat

		var_notAC_candidat.add(tuple(superset.keys())) # Enregistrer tous les supersets qui ne verifient pas les ACi

	return False, var_notAC_candidat


def BC1_4(N_candidats, X, decisions, df=df):
  candidat1=df[df.N_candidat==N_candidats[0]]
  candidat2=df[df.N_candidat==N_candidats[1]]

  # Vérifier que X1 de candidat1 et X2 de candidat2 sont testés sur les memes variables (si ce n'est pas vérifié alors BC3 échoue)
  if X[0].keys()!=X[1].keys():
    return False

  Vmax=[]
  # Générer Vmax : la liste des attributs distincts entre candidat1 et candidat2
  for col in candidat1.columns[1:-1]:
    if candidat1.loc[:,[col]].iloc[0,0]!=candidat2.loc[:,[col]].iloc[0,0]:
      Vmax.append(col)
  #print(Vmax)

  # Vérifier si X est inclu dans Vmax (si ce n'est pas vérifié alors BC3 échoue)
  X_var=[ var for var in X[0]]
  if not set(X_var).issubset(set(Vmax)):
    return False

  # Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
  all_combinaisons_var = []

  for nb_var in range(1, len(Vmax) + 1):
      all_combinaisons_var.extend(combinations(Vmax, nb_var))

  all_combinaisons_var_reversed=all_combinaisons_var[::-1]
  #print(all_combinaisons_var_reversed)

  var_notAC_candidat1=set()
  var_notAC_candidat2=set()

  # Verifier si cet element est partial cause
  for combinaison in tqdm(all_combinaisons_var_reversed):

    if not set(X_var).issubset(combinaison) : # On en veut évaluer que les supersets de X
      continue

    dict_candidat1=candidat1.loc[:,combinaison].iloc[0].to_dict()
    dict_candidat2=candidat2.loc[:,combinaison].iloc[0].to_dict()

    BC1_2_beta_bool1, var_notAC_candidat1=BC1_2_beta2(N_candidats[0], dict_candidat1, decisions[0],var_notAC_candidat1)
    if not (BC1_2_beta_bool1):
      continue

    BC1_2_beta_bool2, var_notAC_candidat2=BC1_2_beta2(N_candidats[1], dict_candidat2, decisions[1], var_notAC_candidat2)
    if not BC1_2_beta_bool2:
      continue


    # Verifier la BC4
    if len(set(X_var)) < len(combinaison):  # BC4 n'est pas vérifiée car nous avons trouver un superset qui verifie BC1_3
      return False,combinaison

    else : # len(X_var)==len(combinaison)
      return True, X_var

  return False, None # Cas ou c'est un sous ensemble de X_var qui vérifie BC1_4


BC1_4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))

 80%|████████  | 204/255 [00:50<00:00, 166.53it/s]

Numéro candidat :  0004



 98%|█████████▊| 249/255 [00:51<00:01,  4.88it/s] 

AC2 et AC3 est vérifié pour une affectation x_prime: {'BDD': 1, 'Logique': 1}
La actual cause est: {'BDD': 0, 'Logique': 0} 



(True, ['Logique'])

In [80]:
BC1_4(("0001","0004"), ({"Logique":"1", "BDD":"1"}, {"Logique":"0", "BDD":"0" }), ('A', 'R')) # le subset qui verifie est ({"Logique":"1"}, {"Logique":"0"}))


100%|██████████| 255/255 [00:33<00:00,  7.66it/s] 


(False, None)